# Building cylindrical configurations with dislocations

![](dislocation_cylinder.png)

The `matscipy.dislocation` provides a set of tools to create atomistic configurations containing dislocations. In this example we focus on cylindrical configurations. In order to create a dislocation we start with a cylinder of bulk material oriented along the vector of the future dislocation line $\vec l$. By convention assumed in `matscipy.dislocation` Z direction of the cell always corresponds to the dislocation line $\vec l$. Then a displacement field is applied to the cylinder and outer radial shell is fixed in order to stabilise the configuration. So we end up with periodic boundary conditions along the dislocation line (Z) and fixed boundaries along X and Y directions. As shown on the figure above, screw dislocations are characterised by a displacement with corresponding burgers vector $\vec b$ parallel to the dislocation line $\vec l$, while edge dislocations have the burgers vector $\vec b$ perpendicular to $\vec l$. `matscipy` uses an anisotropic solution within [Stroh formalism](https://doi.org/10.1080/14786435808565804) to generate displacement field as implemented in [atomman](https://www.ctcms.nist.gov/potentials/atomman/) package.

For the interactive visualisation of the structures we will use [nglview](https://nglviewer.org/nglview/latest/) which allows viewing atomistic structures interactively within an IPython/Jupyter notebooks. We will also color atoms according to the crystallographic structure identified by [Common Neighbour Analysis algoritm](https://www.ovito.org/docs/current/reference/pipelines/modifiers/common_neighbor_analysis.html) and its [extention for diamond structure](https://www.ovito.org/docs/current/reference/pipelines/modifiers/identify_diamond.html) implemented in [OVITO](https://www.ovito.org/docs/current/introduction.html) package. [OVITO](https://www.ovito.org/docs/current/introduction.html), [atomman](https://www.ctcms.nist.gov/potentials/atomman/) and [nglview](https://nglviewer.org/nglview/latest/) are not part of the default dependencies of `matscipy` and require separate installation. Keep that in mind while running these examples as well test in [`test_dislocation.py`](https://github.com/libAtoms/matscipy/blob/master/tests/test_dislocation.py) since the majority of the tests there require this modules and thus will be skipped if the modules are not found in your system. Please note that for this example we use very small systems (small radius). For production calculations one should do a convergence tests with the system size in order to ensure that fixed boundaries do not affect the studied properties.

In [1]:
import numpy as np
import nglview # this import is necessary for rendering of the 3D view
from visualisation import show_dislocation

## Accessing Key Dislocation Properties
As defined in a dimensionless (alat=1) frame, each kind of dislocation has several key properties which describe the geometric layout of the atomic cell, and also the position and orientation of the dislocation. Such properties are stored in matscipy as attributes of each dislocation class, such that they can be accessed without actually needing to construct a dislocation.

As an example:

In [ ]:
from matscipy.dislocation import BCCMixed111Dislocation

print("For a BCC 1/2<111>{110} Mixed Dislocation")
print()

print("The dislocation is oriented in a cell described by the miller indices: ")
print(BCCMixed111Dislocation.axes)

print("Dimensionless burgers vector is: ", BCCMixed111Dislocation.burgers_dimensionless)

print("Dislocation core will be at fractional coords:\n", 
      BCCMixed111Dislocation.unit_cell_core_position_dimensionless, " within a unit cell")

## Available Dislocation Systems
The available dislocation systems are:

In [2]:
import matscipy.dislocation as disloc_mod
import inspect
# Find all classes in matscipy.dislocation which inherit from 
# the Abstract Base Class CubicCrystalDislocation
for name, attr in disloc_mod.__dict__.items():
    if inspect.isclass(attr):
        if issubclass(attr, disloc_mod.CubicCrystalDislocation) and \
            attr not in [disloc_mod.CubicCrystalDislocation,
                         disloc_mod.CubicCrystalDissociatedDislocation]:
            print(name)

BCCScrew111Dislocation
BCCEdge111Dislocation
BCCMixed111Dislocation
BCCEdge100Dislocation
BCCEdge100110Dislocation
DiamondGlide30degreePartial
DiamondGlide90degreePartial
DiamondGlideScrew
DiamondGlide60Degree
FCCScrewShockleyPartial
FCCEdgeShockleyPartial
FCCScrew110Dislocation
FCCEdge110Dislocation


## Body Centered Cubic 

For the case of BCC structure we take tungsten as an example. In order to create a dislocation configuration one has to provide lattice parameter and elastic constants. It is possible to provide these values from literature. However, if you plan to relax the configuration with an interatomic potential, `matscipy.dislocation` provides a convenient method `get_elastic_constants` to calculate desired properties. We will use and Embedded Atom Potential from [Marinica _et. al._ 2013 paper](http://dx.doi.org/10.1088/0953-8984/25/39/395502) (version EAM4) for the case of tungsten.

In [2]:
from matscipy.dislocation import get_elastic_constants
# the calculator to provide forces and energies from the potential
from matscipy.calculators.eam import EAM
eam_calc = EAM("../../tests/w_eam4.fs")

# the function accepts any ASE type of calculator
alat, C11, C12, C44 = get_elastic_constants(calculator=eam_calc, symbol="W", verbose=False)

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 15:29:59      -17.792788*       1.5616
FIRE:    1 15:29:59      -17.796007*       1.2330
FIRE:    2 15:29:59      -17.798904*       0.6354
FIRE:    3 15:29:59      -17.799853*       0.2223
FIRE:    4 15:29:59      -17.799882*       0.1976
FIRE:    5 15:29:59      -17.799928*       0.1509
FIRE:    6 15:29:59      -17.799970*       0.0870
FIRE:    7 15:29:59      -17.799991*       0.0130
FIRE:    8 15:29:59      -17.799981*       0.0629
FIRE:    9 15:29:59      -17.799981*       0.0611
FIRE:   10 15:29:59      -17.799982*       0.0576
FIRE:   11 15:29:59      -17.799984*       0.0525
FIRE:   12 15:29:59      -17.799986*       0.0458
FIRE:   13 15:29:59      -17.799988*       0.0379
FIRE:   14 15:29:59      -17.799989*       0.0289
FIRE:   15 15:29:59      -17.799991*       0.0191
FIRE:   16 15:29:59      -17.799991*       0.0076
FIRE:   17 15:29:59      -17.799992*       0.0052
FI

In [3]:
print(f"{alat:.3f} (Angstrom), {C11:.2f}, {C12:.2f}, {C44:.2f} (GPa)")

3.143 (Angstrom), 538.90, 219.93, 178.80 (GPa)


The values are in good agreement with corresponding numbers in the paper: 3.143, 523, 202, 161

### 1/2<111>{110} screw dislocation

The notation 1/2<111>{110} means a dislocation with a burgers vector 1/2<111> and glide plane {110}. For screw dislocation, the dislocation line is parallel to the burgers vector, thus Z direction of the cell will be along <111>. The glide plane {110} fixes the Y direction to be <110> and the X direction is fixed by the geometry to be along <112>. However you do not have to think about it to start since `BCCScrew111Dislocation` object contains all the necessary information. All you need to do is to pass $a_0$ and elastic constants $C_{11}$, $C_{12}$ and $C_{44}$ and the symbol of the desired BCC element to the constructor.

In [4]:
from matscipy.dislocation import BCCScrew111Dislocation

W_screw = BCCScrew111Dislocation(alat, C11, C12, C44, symbol="W")

print("Cell orientation:")
print(W_screw.axes)

print("Burgers vector:")
print(W_screw.burgers)


Cell orientation:
[[ 1  1 -2]
 [-1  1  0]
 [ 1  1  1]]
Burgers vector:
[1.57169453 1.57169453 1.57169453]


We are going to use `BCCScrew111Dislocation.build_cylinder()` method to build the cells. All the necessary parameters are passed automatically and we only need to chose the radius of the cell in (X,Y) plane. The function returns reference bulk cell and a cell containing dislocation. The displacement field is applied iteratively until a converged value is achieved, the printed output shows the difference between the steps. The radius provided is a parameter is a radius of unconstrained region around the core. The outer shell of fixed atoms is added on top of this. The default value is 10 Angstrom which is about two times larger than the cutoff for the used interatomic potential. The information about which atoms are fixed is stored in `fix_mask` array and can be accessed via `W_screw_dislo.get_array('fix_mask')`.

In [31]:
W_screw_bulk, W_screw_dislo = W_screw.build_cylinder(radius=20)

print("\nCell vectors:")
print(W_screw_dislo.cell.array)

print(f"\nBurgers vector lenth: {np.linalg.norm(W_screw.burgers):.2f} Angstrom")

disloc SCF 0 |d1-d2|_inf = 0.001400723332078968
disloc SCF 1 |d1-d2|_inf = 6.704394990092455e-06
disloc SCF 2 |d1-d2|_inf = 3.2089362583809233e-08

Cell vectors:
[[61.59759392  0.          0.        ]
 [ 0.         62.235928    0.        ]
 [ 0.          0.          2.72225477]]

Burgers vector lenth: 2.72 Angstrom


Now we can look at the created structure with the interactive view. Hovering your mouse over an atom will show chemical symbol of the atom and the structure identified by CNA algorithm. Dislocation is shown with an arrow corresponding to the dislocation line vector $\vec l$. Hovering over the dislocation will display the dislocation name. Moreover, you can use your mouse to control the view:

- Translation: right click + drag
- Rotation: left click + drag
- Z-axis rotation: Ctrl + right click + drag
- Zoom: scroll wheel
- Center view: left click on the desired atom

In [32]:
show_dislocation(W_screw_dislo, d_name="1/2<111> screw")

NGLWidget()

It can be seen that most of the cell is identified as BCC structure with some grey atoms identified as _Other_. These atoms represent the defect atoms, where the algorithm could not assign a crystallographic structure. In this case we have defect atoms at the dislocation core and on the artificial free surfaces on the edges of the cell. The atoms at the _surface_ are part of fixed region of the cell. 

Here we have a cell with the length of one Burgers vector $|\vec b| = \frac{\sqrt{3}}{2}a_0$ along the dislocation line. With the periodic boundary conditions along Z, it corresponds to infinitely long straight dislocation line. Note that the periodic boundary conditions along the dislocation line are not applied in the visualisation. It makes it easier to see the displacement, but you might notice the atoms outside the simulation box. 

To increase the size of the cell along the line we can rely on the magic of [ASE](https://wiki.fysik.dtu.dk/ase/) and simply multiply the cell.

In [38]:
longer_W_screw_dislo = W_screw_dislo * [1, 1, 10]
show_dislocation(longer_W_screw_dislo, d_name="1/2<111> screw")

NGLWidget()

If you want to save the configuration in a file for further analysis or as an input for a code you can use build in [ASE write method](https://wiki.fysik.dtu.dk/ase/ase/io/io.html#ase.io.write) as `longer_W_screw_dislo.write(filename)`. The method supports a wide range of formats including `xyz` and LAMMPS data files. Do not forget to take into account fixed atoms if you use the saved file as an input for a simulation code like LAMMPS. 

### 1/2<111>{110} edge dislocation

As we said before, for edge dislocations burgers vectpr $\vec b$ is perpendicular to the dislocation $\vec l$. So here, we have the same glide plane of (110) which fixes the cell Y direction to <110>. X direction now will be along burgers vector <111> and Z dislocation line direction <112>.

In [7]:
from matscipy.dislocation import BCCEdge111Dislocation

W_edge = BCCEdge111Dislocation(alat, C11, C12, C44, symbol="W")

print("Cell orientation:")
print(W_edge.axes)

print("Burgers vector:")
print(W_edge.burgers)


Cell orientation:
[[ 1  1  1]
 [ 1 -1  0]
 [ 1  1 -2]]
Burgers vector:
[1.57169453 1.57169453 1.57169453]


In [34]:
W_edge_bulk, W_edge_dislo = W_edge.build_cylinder(radius=15)

print("\nCell vectors:")
print(W_edge_dislo.cell.array)

print(f"\nBurgers vector lenth: {np.linalg.norm(W_edge.burgers):.2f} Angstrom")

disloc SCF 0 |d1-d2|_inf = 0.17503717189385382
disloc SCF 1 |d1-d2|_inf = 0.030669651063355258
disloc SCF 2 |d1-d2|_inf = 0.004376489640269432
disloc SCF 3 |d1-d2|_inf = 0.0008807112000284167
disloc SCF 4 |d1-d2|_inf = 0.00015452837692353505
disloc SCF 5 |d1-d2|_inf = 2.6112647029508107e-05
disloc SCF 6 |d1-d2|_inf = 5.7487804006051135e-06
disloc SCF 7 |d1-d2|_inf = 7.771108773702018e-07

Cell vectors:
[[54.44509545  0.          0.        ]
 [ 0.         53.34508114  0.        ]
 [ 0.          0.          7.69969924]]

Burgers vector lenth: 2.72 Angstrom


It can be seen from the print output of ` W_edge.build_cylinder()` that the case of edge dislocation the requires more iterations to achieve converged displacement field.

In [33]:
show_dislocation(W_edge_dislo, scale=0.25, d_name="1/2<111> edge")

NGLWidget()

### 1/2<111>{110} mixed dislocation

For mixed dislocation the cell vector are the same as for the screw dislocation. The difference is that the displacement vector is applied along <111> direction that is not parallel to the Z direction: [$1\bar 11$] and [$1\bar 1 \bar1$] with an angle of 70.5 degrees between the vectors. This leads to both edge and screw component in the displacement and thus the name _mixed_ dislocation.

In [9]:
from matscipy.dislocation import BCCMixed111Dislocation

W_mixed = BCCMixed111Dislocation(alat, C11, C12, C44, symbol="W")

print("Cell orientation:")
print(W_mixed.axes)

print("Burgers vector:")
print(W_mixed.burgers)


Cell orientation:
[[ 1 -1 -2]
 [ 1  1  0]
 [ 1 -1  1]]
Burgers vector:
[ 1.57169453 -1.57169453 -1.57169453]


In [10]:
W_mixed_bulk, W_mixed_dislo = W_mixed.build_cylinder(radius=20)

print("\nCell vectors:")
print(W_mixed_dislo.cell.array)

print(f"\nBurgers vector length: {np.linalg.norm(W_mixed.burgers):.2f} Angstrom")

show_dislocation(W_mixed_dislo, scale=0.5, d_name="1/2<111> mixed")

disloc SCF 0 |d1-d2|_inf = 0.10539187767041772
disloc SCF 1 |d1-d2|_inf = 0.018105716735450528
disloc SCF 2 |d1-d2|_inf = 0.002853744720166418
disloc SCF 3 |d1-d2|_inf = 0.00044617599789692486
disloc SCF 4 |d1-d2|_inf = 6.968641396387643e-05
disloc SCF 5 |d1-d2|_inf = 1.0882499856867955e-05
disloc SCF 6 |d1-d2|_inf = 1.6994203833053945e-06
disloc SCF 7 |d1-d2|_inf = 2.653822296916353e-07

Cell vectors:
[[61.59759392  0.          0.        ]
 [ 0.         62.235928    0.        ]
 [ 0.          0.          2.72225477]]

Burgers vector length: 2.72 Angstrom


NGLWidget()

### <100>{110} edge 'junction' dislocation

So called junction dislocations with burdgers vector along <100> can be formed in the reactions between more abundant dislocations with burgers vector 1/2<111> such as: 

$$
    \frac{1}{2}[1\bar{1}1] + \frac{1}{2}[11\bar{1}] = [100]
$$

They share the same glide plane ad 1/2<111> dislocations and can play important role in impurity segregation. 

In [11]:
from matscipy.dislocation import BCCEdge100110Dislocation

W_100110_edge = BCCEdge100110Dislocation(alat, C11, C12, C44, symbol="W")

print("Cell orientation:")
print(W_100110_edge.axes)

print("Burgers vector:")
print(W_100110_edge.burgers)


Cell orientation:
[[ 1  0  0]
 [ 0  1  1]
 [ 0 -1  1]]
Burgers vector:
[3.14338905 0.         0.        ]


In [12]:
W_100110_edge_bulk, W_100110_edge_dislo = W_100110_edge.build_cylinder(radius=17)
show_dislocation(W_100110_edge_dislo, scale=0.3, 
                 d_name="<100>{110} edge", d_color=[1, 0, 1])

disloc SCF 0 |d1-d2|_inf = 0.256011923011219
disloc SCF 1 |d1-d2|_inf = 0.06016875789278883
disloc SCF 2 |d1-d2|_inf = 0.008688239875985548
disloc SCF 3 |d1-d2|_inf = 0.002036859690120879
disloc SCF 4 |d1-d2|_inf = 0.00025500641555943615
disloc SCF 5 |d1-d2|_inf = 8.302194247189476e-05
disloc SCF 6 |d1-d2|_inf = 1.6579146270331857e-05
disloc SCF 7 |d1-d2|_inf = 1.7272275882884713e-06
disloc SCF 8 |d1-d2|_inf = 6.753932630942927e-07


NGLWidget()

### <100>{001} edge dislocation

This is the same junction dislocation but lying in a different glide plane: (001).

In [13]:
from matscipy.dislocation import BCCEdge100Dislocation

W_100_edge = BCCEdge100Dislocation(alat, C11, C12, C44, symbol="W")

print("Cell orientation:")
print(W_100_edge.axes)

print("Burgers vector:")
print(W_100_edge.burgers)


Cell orientation:
[[ 1  0  0]
 [ 0  0 -1]
 [ 0  1  0]]
Burgers vector:
[3.14338905 0.         0.        ]


In [14]:
W_100_edge_bulk, W_100_edge_dislo = W_100_edge.build_cylinder(radius=20)
show_dislocation(W_100_edge_dislo, scale=0.4, d_name="<100>{001} edge", d_color=[1, 0, 1])

disloc SCF 0 |d1-d2|_inf = 0.29723896079013645
disloc SCF 1 |d1-d2|_inf = 0.03893895819760007
disloc SCF 2 |d1-d2|_inf = 0.011189961805871729
disloc SCF 3 |d1-d2|_inf = 0.001492901691747324
disloc SCF 4 |d1-d2|_inf = 0.00046542738716470744
disloc SCF 5 |d1-d2|_inf = 8.162136711258039e-05
disloc SCF 6 |d1-d2|_inf = 1.3593036747305831e-05
disloc SCF 7 |d1-d2|_inf = 4.370960947186475e-06
disloc SCF 8 |d1-d2|_inf = 5.007713447316431e-07


NGLWidget()

## Face Centered Cubic

As an example of FCC material we will consider Ni and an interatomic potential from [Bonny _et. al._](https://www.tandfonline.com/doi/abs/10.1080/14786430903299824)


In [15]:
eam_calc = EAM("../../tests/FeCuNi.eam.alloy")

# the function accepts any ASE type of calculator
alat, C11, C12, C44 = get_elastic_constants(calculator=eam_calc, symbol="Ni", verbose=False)

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 15:30:04      -17.801046*       0.0001
Fitting C_11
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-5.12800014e+00, -2.50333320e+00, -1.41366167e-04,  2.37961959e+00,
        4.63379565e+00]) GPa
Cij (gradient) / GPa    :     244.0654437182867
Error in Cij / GPa      :     4.217968506243491
Correlation coefficient :     0.9995522932872427
Setting C11 (1) to 1.523337 +/- 0.026326


Fitting C_21
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-3.37539886e+00, -1.58459192e+00, -1.41366167e-04,  1.38619385e+00,
        2.58171226e+00]) GPa
Cij (gradient) / GPa    :     148.85008009740656
Error in Cij / GPa      :     6.776719382842348
Correlation coefficient :     0.9969053432709588
Setting C21 (7) to 0.929049 +/- 0.042297


Fitting C_31
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-3.37539886e+00, -1.58459192e+00, -1.41366167e-04,

In [16]:
print(f"{alat:.3f} (Angstrom), {C11:.2f}, {C12:.2f}, {C44:.2f} (GPa)")

3.520 (Angstrom), 244.07, 148.85, 125.62 (GPa)


### 1/2<110>{111} screw dislocation (perfect and dissociated)

In [17]:
from matscipy.dislocation import FCCScrew110Dislocation

Ni_screw = FCCScrew110Dislocation(alat, C11, C12, C44, symbol="Ni")

print("Cell orientation:")
print(Ni_screw.axes)

print("Burgers vector:")
print(Ni_screw.burgers)


Cell orientation:
[[ 1  1 -2]
 [ 1  1  1]
 [ 1 -1  0]]
Burgers vector:
[ 1.76 -1.76  0.  ]


In [18]:
Ni_screw_bulk, Ni_screw_dislo = Ni_screw.build_cylinder(radius=20)


print("\nCell vectors:")
print(Ni_screw_dislo.cell.array)

print(f"\nBurgers vector length: {np.linalg.norm(Ni_screw.burgers):.2f} Angstrom")

show_dislocation(Ni_screw_dislo, 
                 d_name="1/2<110> screw", 
                 d_color=[0, 0, 1])

disloc SCF 0 |d1-d2|_inf = 0.021791458261984697
disloc SCF 1 |d1-d2|_inf = 0.0002936041882874109
disloc SCF 2 |d1-d2|_inf = 4.8928735380371347e-05
disloc SCF 3 |d1-d2|_inf = 1.2696678054050815e-06
disloc SCF 4 |d1-d2|_inf = 1.0764587210498888e-07
disloc SCF 0 |d1-d2|_inf = 0.020321133865591423
disloc SCF 1 |d1-d2|_inf = 0.0006134137472214689
disloc SCF 2 |d1-d2|_inf = 5.9406489367308524e-05
disloc SCF 3 |d1-d2|_inf = 2.948341477265748e-06
disloc SCF 4 |d1-d2|_inf = 8.340714410803862e-08

Cell vectors:
[[60.35542726  0.          0.        ]
 [ 0.         60.96818843  0.        ]
 [ 0.          0.          2.48901587]]

Burgers vector length: 2.49 Angstrom


NGLWidget()

Due to stable Intrinsic Stacking Fault (ISF) the 1/2<110> dislocations are not stable and dissociate in two 1/6<112> Shockley partials separated by stacking fault. For example:

$$
    \frac{1}{2}[1\bar10] \rightarrow \frac{1}{6}[2\bar1\bar1] + \mathrm{ISF} + \frac{1}{2}[1\bar21]
$$

where ISF is intrinsic stacking fault. It is possible to pass a parameter `partial_distance` to `build_cylinder()` function in order to create dissociated dislocation. `partial distance` defines separation distance (length of the stacking fault) of two partial dislocations. The value corresponds to number of glide distances the distance in Angstrom be obtaines as `patial_distance * dislocation.glide_distance`.

In [35]:
Ni_screw_bulk, Ni_screw_dislo = Ni_screw.build_cylinder(radius=20, partial_distance=5)
print(f"\nExpected partial distance: {5 * Ni_screw.glide_distance:.1f} Angstrom")
show_dislocation(Ni_screw_dislo, 
                 partial_distance=5 * Ni_screw.glide_distance,
                 d_name="1/6<112> Shockley partial screw")

disloc SCF 0 |d1-d2|_inf = 0.02179145826198453
disloc SCF 1 |d1-d2|_inf = 0.0002936041882874109
disloc SCF 2 |d1-d2|_inf = 4.892873538059339e-05
disloc SCF 3 |d1-d2|_inf = 1.2696678051752307e-06
disloc SCF 4 |d1-d2|_inf = 1.0764587193845543e-07
disloc SCF 0 |d1-d2|_inf = 0.020321133865591506
disloc SCF 1 |d1-d2|_inf = 0.0006134137472214689
disloc SCF 2 |d1-d2|_inf = 5.9406489367308524e-05
disloc SCF 3 |d1-d2|_inf = 2.948341477515548e-06
disloc SCF 4 |d1-d2|_inf = 8.340714444804442e-08

Expected partial distance: 10.8 Angstrom


NGLWidget()

Together with FCC (green) and defect atoms (grey) the CNA algorithm identified the atoms of the stacking fault as HCP structure (pink), which is a result of the local change of stacking order of the atomic layers within the defect.

### 1/2<110>{111} edge dislocation (perfect and dissociated)

In [20]:
from matscipy.dislocation import FCCEdge110Dislocation

Ni_edge = FCCEdge110Dislocation(alat, C11, C12, C44, symbol="Ni")

print("Cell orientation:")
print(Ni_edge.axes)

print("Burgers vector:")
print(Ni_edge.burgers)


Cell orientation:
[[ 1 -1  0]
 [ 1  1  1]
 [-1 -1  2]]
Burgers vector:
[ 1.76 -1.76  0.  ]


In [21]:
Ni_edge_bulk, Ni_edge_dislo = Ni_edge.build_cylinder(radius=20)

print("\nCell vectors:")
print(Ni_edge_dislo.cell.array)

print(f"\nBurgers vector length: {np.linalg.norm(Ni_edge.burgers):.2f} Angstrom")

show_dislocation(Ni_edge_dislo, d_name="1/2<110> edge", d_color=[0, 0, 1])

disloc SCF 0 |d1-d2|_inf = 0.04310715442453525
disloc SCF 1 |d1-d2|_inf = 0.0031359890399249857
disloc SCF 2 |d1-d2|_inf = 0.00018078113282300745
disloc SCF 3 |d1-d2|_inf = 1.4501349156037513e-05
disloc SCF 4 |d1-d2|_inf = 9.978781161934513e-07
disloc SCF 0 |d1-d2|_inf = 0.03462421627571943
disloc SCF 1 |d1-d2|_inf = 0.0022123322463467043
disloc SCF 2 |d1-d2|_inf = 0.00021564683900149317
disloc SCF 3 |d1-d2|_inf = 1.1838612354972411e-05
disloc SCF 4 |d1-d2|_inf = 6.324895580611667e-07

Cell vectors:
[[64.71441261  0.          0.        ]
 [ 0.         60.96818843  0.        ]
 [ 0.          0.          4.31110195]]

Burgers vector length: 2.49 Angstrom


NGLWidget()

In [22]:
Ni_edge_bulk, Ni_edge_dislo = Ni_edge.build_cylinder(radius=20, partial_distance=10)
print(f"\nExpected partial distance: {10 * Ni_edge.glide_distance:.1f} Angstrom")
show_dislocation(Ni_edge_dislo, 
                 partial_distance=10 * Ni_edge.glide_distance, 
                 d_name="1/2<110> Shockley partial edge")

disloc SCF 0 |d1-d2|_inf = 0.04310715442453525
disloc SCF 1 |d1-d2|_inf = 0.0031359890399249857
disloc SCF 2 |d1-d2|_inf = 0.00018078113282300745
disloc SCF 3 |d1-d2|_inf = 1.4501349155759957e-05
disloc SCF 4 |d1-d2|_inf = 9.978781162003902e-07
disloc SCF 0 |d1-d2|_inf = 0.03462421627571943
disloc SCF 1 |d1-d2|_inf = 0.002212332246347065
disloc SCF 2 |d1-d2|_inf = 0.00021564683900149317
disloc SCF 3 |d1-d2|_inf = 1.1838612354972411e-05
disloc SCF 4 |d1-d2|_inf = 6.324895580611667e-07
Expected partial distance: 12.4 Angstrom


NGLWidget()

## Diamond Cubic

As an example of diamond structure we will use Si and potential from [work of D. Holland and M. Marder](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.80.746).

In [23]:
from matscipy.calculators.manybody.explicit_forms.stillinger_weber import StillingerWeber,\
                                                                Holland_Marder_PRL_80_746_Si
from matscipy.calculators.manybody import Manybody
calc = Manybody(**StillingerWeber(Holland_Marder_PRL_80_746_Si))

# the function accepts any ASE type of calculator
alat, C11, C12, C44 = get_elastic_constants(calculator=calc, symbol="Si", verbose=False)

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 15:30:06      -34.692786*       0.0922
FIRE:    1 15:30:06      -34.692742*       0.1881
FIRE:    2 15:30:06      -34.692797*       0.0452


FIRE:    3 15:30:06      -34.692771*       0.1323
FIRE:    4 15:30:06      -34.692781*       0.1071
FIRE:    5 15:30:06      -34.692794*       0.0616
FIRE:    6 15:30:06      -34.692800*       0.0043
FIRE:    7 15:30:06      -34.692795*       0.0537
FIRE:    8 15:30:06      -34.692796*       0.0511
FIRE:    9 15:30:06      -34.692796*       0.0461
FIRE:   10 15:30:06      -34.692798*       0.0390
FIRE:   11 15:30:06      -34.692799*       0.0299
FIRE:   12 15:30:06      -34.692799*       0.0195
FIRE:   13 15:30:06      -34.692800*       0.0081
FIRE:   14 15:30:06      -34.692800*       0.0036
FIRE:   15 15:30:06      -34.692800*       0.0036
FIRE:   16 15:30:06      -34.692800*       0.0035
FIRE:   17 15:30:06      -34.692800*       0.0034
FIRE:   18 15:30:06      -34.692800*       0.0032
FIRE:   19 15:30:06      -34.692800*       0.0030
FIRE:   20 15:30:06      -34.692800*       0.0028
FIRE:   21 15:30:06      -34.692800*       0.0025
FIRE:   22 15:30:06      -34.692800*       0.0022


In [24]:
print(f"{alat:.3f} (Angstrom), {C11:.2f}, {C12:.2f}, {C44:.2f} (GPa)")

5.431 (Angstrom), 201.46, 51.64, 118.18 (GPa)


In [25]:
from matscipy.dislocation import DiamondGlideScrew

Si_screw = DiamondGlideScrew(alat, C11, C12, C44, symbol="Si")

print("Cell orientation:")
print(Si_screw.axes)

print("Burgers vector:")
print(Si_screw.burgers)


Cell orientation:
[[ 1  1 -2]
 [ 1  1  1]
 [ 1 -1  0]]
Burgers vector:
[ 2.71547466 -2.71547466  0.        ]


### 1/2<110>{111} screw dislocation (perfect and dissociated)

In [36]:
Si_screw_bulk, Si_screw_dislo = Si_screw.build_cylinder(radius=20)

print("\nCell vectors:")
print(Si_screw_dislo.cell.array)

print(f"\nBurgers vector length: {np.linalg.norm(Si_screw.burgers):.2f} Angstrom")

show_dislocation(Si_screw_dislo, 
                 diamond_structure=True, 
                 scale=0.3, 
                 add_bonds=True, # bonds make it a bit easier to see the structure
                 d_name="1/2<110> screw", 
                 d_color=[0, 0, 1])


Cell vectors:
[[66.51527324  0.          0.        ]
 [ 0.         75.25344122  0.        ]
 [ 0.          0.          3.84026109]]

Burgers vector length: 3.84 Angstrom


NGLWidget()

The modification of the CNA algorithm for the diamond structure identifies the defect atoms gradually depending how _far_ they are from the perfect structure. The important outcome for us is now dislocation core and free surface atoms are identified as 1st and 2nd neighbors of the _Cubic diamond_ structure. 

As in FCC structure similar dissociation mechanism exists in diamond structure.

In [27]:
Si_screw_bulk, Si_screw_dislo = Si_screw.build_cylinder(radius=20, partial_distance=5)

print(f"Expected partial distance: {5 * Si_screw.glide_distance:.1f} Angstrom")
show_dislocation(Si_screw_dislo, 
                 diamond_structure=True, # bonds make it a bit easier to see the structure
                 scale=0.3, add_bonds=True, 
                 partial_distance=5 * Si_screw.glide_distance, 
                 d_name="1/6<112> 30 degree partial screw") 

Expected partial distance: 16.6 Angstrom


NGLWidget()

The stacking fault atoms are identified as _Hexagonal diamond_ structure.

### 1/2<110>{111} 60 degree screw dislocation (perfect and dissociated)

Due to the particular symmetry of cubic diamond structure, there exist dislocations with 60 degree angle between burgers vector and dislocation line. This dislocation can dissociate in two partials: 30 and 90 degree.

In [28]:
from matscipy.dislocation import DiamondGlide60Degree

Si_60_degree_screw = DiamondGlide60Degree(alat, C11, C12, C44, symbol="Si")

print("Cell orientation:")
print(Si_screw.axes)

print("Burgers vector:")
print(Si_screw.burgers)


Cell orientation:
[[ 1  1 -2]
 [ 1  1  1]
 [ 1 -1  0]]
Burgers vector:
[ 2.71547466 -2.71547466  0.        ]


In [29]:
Si_60_degree_screw_bulk, \
Si_60_degree_screw_dislo = Si_60_degree_screw.build_cylinder(radius=20)

print("\nCell vectors:")
print(Si_60_degree_screw_dislo.cell.array)

print(f"\nBurgers vector length: {np.linalg.norm(Si_60_degree_screw.burgers):.2f} Angstrom")

show_dislocation(Si_60_degree_screw_dislo, 
                 diamond_structure=True, 
                 scale=0.3, add_bonds=True,
                 d_name="1/2<110> 60 degree screw", 
                 d_color=[0, 0, 1])


Cell vectors:
[[66.51527324  0.          0.        ]
 [ 0.         75.25344122  0.        ]
 [ 0.          0.          3.84026109]]

Burgers vector length: 3.84 Angstrom


NGLWidget()

In [30]:
Si_60_degree_screw_bulk, \
Si_60_degree_screw_dislo = Si_60_degree_screw.build_cylinder(radius=20, 
                                                             partial_distance=5)

print(f"Expected partial distance: {5 * Si_60_degree_screw.glide_distance:.1f} Angstrom")
show_dislocation(Si_60_degree_screw_dislo, 
                 diamond_structure=True, 
                 scale=0.3, 
                 add_bonds=True, 
                 d_color=[1, 0, 1],
                 partial_distance=5 * Si_60_degree_screw.glide_distance, 
                 d_name=["1/6<112> 90 degree partial screw", 
                         "1/6<112> 30 degree partial screw"])

Expected partial distance: 16.6 Angstrom


NGLWidget()